## 10fps로 바꾸는 것까지 진행 완료

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import time

In [2]:
mp_hands = mp.solutions.hands
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

os.makedirs('Datasets/', exist_ok=True)

In [3]:
study_frame=500 # 500이면 30fps 기준으로 17초
FPS = 10
prev_time = 0
hand="Right" 
pause_time=5
gesture_list = ['NONE', 'ULF', 'UMF', 'URF', 'DLF', 'DMF', 'DRF', 'ULO', 'URO', 'DLO', 'DRO', 'ULB', 'UMB', 'URB', 'DLB', 'DMB', 'DRB', 'TON']
gesture=gesture_list[1]

In [5]:
cap = cv2.VideoCapture(0)
        
position = np.zeros((1,21,3))
velocity = np.zeros((1,21,3))
acceleration = np.zeros((1,21,3))
res=np.zeros((1,21,3))

with mp_hands.Hands(model_complexity=1,
                    max_num_hands=1,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5) as hands:
    
    

    print('::{} {} Ready.... '.format(hand,gesture))
    cnt=pause_time
    
    for i in [1000]*pause_time:
        print(cnt)
        cv2.waitKey(i)
        cnt-=1

    print('Start')

    while len(position) <= study_frame:

        success, frame = cap.read()
        current_time = time.time() - prev_time
        
        if not success:
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame,hand_landmarks,mp_hands.HAND_CONNECTIONS,
                                          mp_drawing_styles.get_default_hand_landmarks_style(),
                                          mp_drawing_styles.get_default_hand_connections_style())

        frame_vector_x = []
        frame_vector_y = []
        frame_vector_z = []
        if results.multi_hand_landmarks:
            for point in results.multi_hand_landmarks[0].landmark:
                frame_vector_x.append(point.x)
                frame_vector_y.append(point.y)
                frame_vector_z.append(point.z)
            frame_array = np.vstack((frame_vector_x,frame_vector_y,frame_vector_z)).reshape(1,21,3)

            position = np.vstack((position,frame_array))
        
            if len(position)>2:
                current_velocity = position[-1] - position[-2]
                velocity=np.vstack((velocity, current_velocity.reshape(1,21,3)))

            if len(position)>3:
                current_acceleration = velocity[-1] - velocity[-2]
                acceleration=np.vstack((acceleration, current_acceleration.reshape(1,21,3)))

        if (success is True) and (current_time > 1./FPS):
            prev_time = time.time()
            cv2.putText(frame,'Dataset Count : %d' % (position.shape[0]-1),(20,40),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0))
            cv2.imshow('frame', frame)

            if cv2.waitKey(1) == ord('q'):
                break

    position=position[1:]
    velocity=velocity[1:]
    acceleration=acceleration[1:]
 

    np.save('Datasets/%s_%s_p' % (hand,gesture),position)
    np.save('Datasets/%s_%s_v' % (hand,gesture),velocity)
    np.save('Datasets/%s_%s_a' % (hand,gesture),acceleration)
    cv2.waitKey(1000)


cap.release()
cv2.destroyAllWindows()
print('Finished')

::Right ULF Ready.... 
5
4
3
2
1
Start
Finished


In [7]:
print('::Brushing teeth  Procedure::')
print(hand, end='\t')
for gesture in gesture_list:
    if gesture==gesture_list[-1]:
        print(gesture, end='')
    else:
        print(gesture, end='')
        print('->', end='')
print()

for gesture in gesture_list:
    cap = cv2.VideoCapture(0)

    position = np.zeros((1,21,3))
    velocity = np.zeros((1,21,3))
    acceleration = np.zeros((1,21,3))
    res=np.zeros((1,21,3))

    with mp_hands.Hands(model_complexity=1,
                        max_num_hands=1,
                        min_detection_confidence=0.5,
                        min_tracking_confidence=0.5) as hands:



        print('::{} {} Ready.... '.format(hand,gesture))
        cnt=pause_time

        for i in [1000]*pause_time:
            print(cnt)
            cv2.waitKey(i)
            cnt-=1

        print('Start')

        while len(position) <= study_frame:

            success, frame = cap.read()
            current_time = time.time() - prev_time
            
            if not success:
                break

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame)
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(frame,hand_landmarks,mp_hands.HAND_CONNECTIONS,
                                              mp_drawing_styles.get_default_hand_landmarks_style(),
                                              mp_drawing_styles.get_default_hand_connections_style())

            frame_vector_x = []
            frame_vector_y = []
            frame_vector_z = []
            if results.multi_hand_landmarks:
                for point in results.multi_hand_landmarks[0].landmark:
                    frame_vector_x.append(point.x)
                    frame_vector_y.append(point.y)
                    frame_vector_z.append(point.z)
                frame_array = np.vstack((frame_vector_x,frame_vector_y,frame_vector_z)).reshape(1,21,3)

                position = np.vstack((position,frame_array))

                if len(position)>2:
                    current_velocity = position[-1] - position[-2]
                    velocity=np.vstack((velocity, current_velocity.reshape(1,21,3)))

                if len(position)>3:
                    current_acceleration = velocity[-1] - velocity[-2]
                    acceleration=np.vstack((acceleration, current_acceleration.reshape(1,21,3)))

            if (success is True) and (current_time > 1./FPS):
                prev_time = time.time()
                cv2.putText(frame,'Dataset Count : %d' % (position.shape[0]-1),(20,40),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0))
                cv2.imshow('frame', frame)

                if cv2.waitKey(1) == ord('q'):
                    break

        position=position[1:]
        velocity=velocity[1:]
        acceleration=acceleration[1:]


        np.save('Datasets/%s_%s_p' % (hand,gesture),position)
        np.save('Datasets/%s_%s_v' % (hand,gesture),velocity)
        np.save('Datasets/%s_%s_a' % (hand,gesture),acceleration)
        cv2.waitKey(1000)


    cap.release()
    cv2.destroyAllWindows()
    print('Finished')

::Brushing teeth  Procedure::
Right	NONE->ULF->UMF->URF->DLF->DMF->DRF->ULO->URO->DLO->DRO->ULB->UMB->URB->DLB->DMB->DRB->TON
::Right NONE Ready.... 
5
4
3
2
1
Start
Finished
::Right ULF Ready.... 
5
4
3
2
1
Start


KeyboardInterrupt: 